In [1]:

import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_chroma import Chroma
import os
import time
import pandas as pd
import json


In [ ]:
# Load the CSV file into a DataFrame
path = "PREPARE_Datasets_Hatebase/"
file_path = path + "FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx"
df = pd.read_excel(file_path)
len(df)
df_test = df


In [5]:
pipeline_number = '1' #you can use 1,2,3 etc.. (defined in pipeline.py)

pipeline = {
    "pipeline": f"pipeline_{pipeline_number}"
}

# Save the variables to a JSON file
with open('choose_pipeline.json', 'w') as f:
    json.dump(pipeline, f)


# Define the variables
variables = {
    "GROQ_API_KEY": "<KEY>",
    "OPENAI_API_KEY": "<KEY>",
    "use_openai": True,
    "model_name": "all-MiniLM-L6-v2.gguf2.f16.gguf",
    "collection_name": "MA_GPT4",
    "persist_directory": "./db_GPT4",
    "chat_model": "gpt-3.5-turbo-0125", #mixtral-8x7b-32768", # Add your desired model
    "model": "gpt-3.5-turbo-0125", # Add your desired model
    "output_path_experiments": "./experiments_outputs/",
    "content_used": "EN_classify_reply_comment_title_definition_description_descriptionYT_transcript_audio_event_object",  # Add this variable or adjust as needed
    
}


# Save the variables to a JSON file
with open(f'variables{pipeline_number}.json', 'w') as f:
    json.dump(variables, f)

In [ ]:
content_used = variables["content_used"]
content_used 

In [ ]:
df_input = df_test#[155:]
df_input.comment.iloc[0]

In [ ]:
file_name = f"{content_used}_classify_replies_561_GPT35T_Hatebase_1"
output_file = f"./experiments_outputs/{file_name}.csv"
print(output_file)

variables_logging = {
    "approach_name":  file_name 
}

# Save the variables to a JSON file
with open(f'settings_logging.json', 'w') as f:
    json.dump(variables_logging, f)

    

In [ ]:

import pipeline as pipe
os.environ.get("<KEY>")

# Example usage
row_index = None #66#None #3  # Set to None to process all rows
use_predefined = False#True  # Set to True to use predefined context and reply for the specific index
#content_used = "EN_reply_comment_title_transcript" #"title_transcript_comment_reply"
content_used = variables["content_used"]
language ="EN"


# output_path_experiments = "./experiments_outputs/"
# output_file = output_path_experiments + f'{content_used}_classify_replies_800_no_duplicates.csv'

# predefined_comment = "Was geht mit den Ü30 ab? 😂 Diese Probleme hätte ich auch gerne, sich einfach angegriffen zu fühlen"
# predefined_reply = "😂 machmal tut es weh"


''' You can choose between iterating through a whole dataset/dataframe or by only using one specific row by index. 
Also, if only one row is used a custom comment and reply text can be used since these are currently not defined in the dataset. 
(Therefore the iterating approach is currently not working)'''
custom_entities = []

#classes = ["abusive", "non-abusive", "-uncertain or borderline"]
classes = ["appropriate", "inappropriate", "offensive", "violent"]
response_df = pipe.main_pipeline(df_input, classes, content_used, custom_entities, language, index=row_index, use_wikipedia_directly=True, use_attention_score=False, use_spacy=True, use_propn=False, use_keywords=False, predefined_context=predefined_comment if use_predefined else None, predefined_reply=predefined_reply if use_predefined else None)

response_df.to_csv(output_file, sep = ';', index=None)